## Making a lexicon in this notebook that suits our need

**For our lexicon we need a list of all the vocabulary words we will use. However we need to take some things into consideration as well**

- We need to make sure our lexicon includes only lemmatized words.
- Because of this, when we will be using our word, we have to make sure we parse it lemmatized as well.
- This would be hard considering we would have to take into account which part of speech that specific word is.
- A better alternative approach would be to have the same wordID for different forms of words.
- Achieving this can be very hard with a single list.
- We need to think of a better data structure that can help us assign wordIDs to different words.
- The method to do that is still being worked on :)

In [1]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.corpus import words as wordcorpus
import lexicon_proto_file_pb2 as lexproto
stop_words = set(stopwords.words('english'))

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### Method 1:

- A method we can use is to make 4-5 alternate lists of same size to store different kinds of words.
- Each of these lists will have the same index, but different words:

| Index | List1 | List2 | List3 | List4 |
| --- | --- | --- | --- | --- |
| 0 | Think | Thinks | Thought | Thinking |
| 1 | run | runs | ran | running |
| 2 | super | "" | "" | "" |


#### First of all we will make our List1

In order to make our first list, we will get 4 types of words:

- Verbs
- Nouns
- Adverbs
- Adjectives


We will make this function that gets each kind of word from wordnet in their simple forms
Also will remove stopwords from it

In [2]:
def get_simple_forms(pos):
    words = set()
    for synset in list(wordnet.all_synsets(pos=pos)):
        for lemma in synset.lemmas():
            words.add(lemma.name().lower())
    return list(words)

# getting the 4 types of words
simple_all = []
for pos in ['v', 'n', 'a', 'r']:
    # Getting a list of all types of words
    word_list = get_simple_forms(pos)
    # making sure all the stopwords are removed from here
    word_list = [word for word in word_list if word not in stop_words]
    # word_list = [lemmatizer.lemmatize(word, pos) for word in word_list]  || This is not needed as its already in simple form
    simple_all.append(word_list)
    print(len(word_list))




11517
117751
21443
4442


In [3]:
all_words = []
for i in range(4):
    print(simple_all[i][:5])
    all_words += simple_all[i]

print("Size of words before removing duplicates: ", len(all_words))
all_words = list(set(all_words))
print("Size of words after removing duplicates: ", len(all_words))


['mellow_out', 'propel', 'break_out', 'examine', 'abort']
['common_dolphin', 'narcotics_agent', 'meadow_goldenrod', 'curability', 'green_pepper']
['coal-black', 'intramolecular', 'bleak', 'patient_of', 'aphanitic']
['unjustifiably', 'mawkishly', 'de_jure', 'in_one_ear', 'captiously']
Size of words before removing duplicates:  155153
Size of words after removing duplicates:  147229


### Now we will get a list of all possible words

In [4]:
all_words_extra = wordcorpus.words()
print(all_words_extra[:10])
print(all_words_extra.__len__())


['A', 'a', 'aa', 'aal', 'aalii', 'aam', 'Aani', 'aardvark', 'aardwolf', 'Aaron']
236736


We will be using our Lemmatizer now to check which form matches any word from the list

## Scratching method 1 as it is overcomplicated. 

A simpler method would be to keep the original list and when a new word comes, we will use all 4 kind of lemmatizers to convert it into 4 types of words and find either of them in the list

The 4 types we will parse in this order: n -> a -> v -> r

**Hence, our lexicon is final, just need to add the stopwords at the start**

In [5]:
stop_words

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [6]:
print(len(list(stop_words)))
print(len(all_words))

179
147229


**Adding 179 Stop words at the start of this list**

We need to separate these words as they occur very frequently in documents.
Completely removing them is not feasible and a better option would be to add them to the start and reduce t

In [7]:
all_words = list(stop_words) + all_words
len(all_words)

147408

#### I will make  a proto buffer file to keep this data safe now, and to be used later

In [9]:
proto_wordlist = lexproto.Lexicon()
proto_wordlist.wordlist.extend(all_words)


fileData = proto_wordlist.SerializeToString()
print(fileData)
with open("lexicon.pb", "wb") as file:
    file.write(fileData)

b'\n\x08mightn\'t\n\x04hadn\n\x06mightn\n\x04more\n\x04with\n\x03who\n\x04hasn\n\x04that\n\x05below\n\x04such\n\x01m\n\x04over\n\x06should\n\x07between\n\x02as\n\x04what\n\x04didn\n\x05needn\n\x07weren\'t\n\x01d\n\x02ma\n\x04shan\n\x02on\n\x02we\n\x07himself\n\x05about\n\x06itself\n\x02or\n\x05while\n\x04both\n\x02is\n\x02ve\n\x05isn\'t\n\x06having\n\x04aren\n\x02me\n\x04they\n\x03too\n\x03out\n\x01t\n\x06couldn\n\x05there\n\x05other\n\x07needn\'t\n\x05above\n\x03not\n\tshouldn\'t\n\x05you\'d\n\x06aren\'t\n\x02my\n\x02so\n\x06myself\n\x07further\n\x04only\n\x01i\n\x04down\n\x03how\n\x07herself\n\x04most\n\x04will\n\x06you\'ve\n\x02by\n\x02an\n\x03was\n\x03for\n\x02he\n\x06wouldn\n\x02in\n\x02at\n\x07that\'ll\n\x08couldn\'t\n\x06theirs\n\x02re\n\x05won\'t\n\x05mustn\n\x06didn\'t\n\x02am\n\x03off\n\x06hasn\'t\n\tourselves\n\x02be\n\x05after\n\x04then\n\x03few\n\x05until\n\x06hadn\'t\n\x04your\n\x04some\n\x05weren\n\x04once\n\x03ain\n\x06shan\'t\n\x03its\n\x03our\n\x04from\n\x04into\n\x07

In [26]:
proto_wordlist.wordlist[16]

'didn'